In [105]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim


In [106]:
from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']

# The ID and range of a sample spreadsheet.
SAMPLE_SPREADSHEET_ID = '1TXvsECM537MMj1nVL-wcpXbSUbOj2-S6JVndcb2mOTE'
SAMPLE_RANGE_NAME = 'New typeform!A:Z'

def refresh():
    """Shows basic usage of the Sheets API.
    Prints values from a sample spreadsheet.
    """
    creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server()
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('sheets', 'v4', credentials=creds)

    # Call the Sheets API
    sheet = service.spreadsheets()
    result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                range=SAMPLE_RANGE_NAME).execute()
    values = result.get('values', [])
    df = pd.DataFrame(values[1:],columns=values[0])
    df['Submitted At']=pd.to_datetime(df['Submitted At'])
    
# from pyproj import Proj, transform 
# geolocator = Nominatim(user_agent="none")
# def LongLat_to_EN(long, lat):
#         easting, northing = transform(
#         Proj(init='epsg:4326'), Proj(init='epsg:3857'), long, lat)
#         return easting, northing
    lats,longs=[],[]  

    for loc in df["Where do you live? Give me your postcode, please"]:
        try:
            location = geolocator.geocode(loc)#change
            lats.append(location.latitude)
            longs.append(location.longitude)
        except:
            lats.append(np.nan)
            longs.append(np.nan)

    # coordsEN=[LongLat_to_EN(longs[i],lats[i]) for i in range(len(df))]
    # x = [coordsEN[i][0] for i in range(len(coordsEN))]
    # y =[ coordsEN[i][1] for i in range(len(coordsEN))]
    df['x'],df['y']=longs,lats



    
    return df


In [104]:
from geopy.geocoders import Nominatim
# from pyproj import Proj, transform 
# geolocator = Nominatim(user_agent="none")
# def LongLat_to_EN(long, lat):
#         easting, northing = transform(
#         Proj(init='epsg:4326'), Proj(init='epsg:3857'), long, lat)
#         return easting, northing
lats,longs=[],[]  

for loc in df["Where do you live? Give me your postcode, please"]:
    try:
        location = geolocator.geocode(loc)
        lats.append(location.latitude)
        longs.append(location.longitude)
    except:
        lats.append(np.nan)
        longs.append(np.nan)

# coordsEN=[LongLat_to_EN(longs[i],lats[i]) for i in range(len(df))]
# x = [coordsEN[i][0] for i in range(len(coordsEN))]
# y =[ coordsEN[i][1] for i in range(len(coordsEN))]
df['x'],df['y']=longs,lats




In [ ]:
geolocator = Nominatim(user_agent="none")
for port in au.index:
    location = geolocator.geocode(port+ ', Australia')
    lats.append(location.latitude)
    longs.append(location.longitude)
        
def LongLat_to_EN(long, lat):
        easting, northing = transform(
        Proj(init='epsg:4326'), Proj(init='epsg:3857'), long, lat)
        return easting, northing
coordsEN=[LongLat_to_EN(longs[i],lats[i]) for i in range(len(au))]
x = [coordsEN[i][0] for i in range(len(coordsEN))]
y =[ coordsEN[i][1] for i in range(len(coordsEN))]

In [130]:
contact = pd.read_csv('contact.csv')
import dash_table

app.layout = dash_table.DataTable(
    id='table',
    columns=[{"name": i, "id": i} for i in contact.columns],
    data=contact.to_dict("rows"),
)

ModuleNotFoundError: No module named 'dash_table'

In [148]:
import dash
from dash.dependencies import Input, Output
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go
    

image_filename = '/Users/Adil/Documents/Data Science/NLTK/wc.png'
df = refresh()
app = dash.Dash()

options = [{'label': i, 'value': i} for i in df["What's your name?"].unique()]

    
app.layout = html.Div([
    
    html.H1(
        '360 Degrees'
    ),
    
    html.Div(
        'hkjhl'
    ),
    
    html.Div([
        dcc.Dropdown(
            id='my-dropdown1',
            options=options,
            multi=False,
            placeholder='Please select a client'
    ),
        dcc.Dropdown(
            id='my-dropdown2',
            options=options,
            multi=False,
            placeholder='Please select a client'
    ),
        
        
        
    ]
        
        , style={'width': '20%', 'display': 'inline-block','vertical-align': 'middle'}),    

    html.H3(
        'Hours Spent Researching.'
    ),
    
    
    html.Div(
        id='my-graph1'
    , style={'width': '50%', 'display': 'inline-block','vertical-align': 'right'})

])



@app.callback(
    Output(component_id='my-graph1', component_property='children'),
    [Input(component_id='my-dropdown1', component_property='value'),
    ]
)

def update_graph(selected_dropdown_value):
    df=refresh()
    try:
        data=[go.Scatter(x=df[df["What's your name?"]==selected_dropdown_value]['Submitted At'],
                         y=df[df["What's your name?"]==selected_dropdown_value][df.columns[2]],
                        )]
    
        return     dcc.Graph(id='outgraph1',
            figure=go.Figure(data=data,
            layout=go.Layout(showlegend=True,margin= {'l': 10, 'b': 20, 't': 0, 'r': 0})
  
                            ))
    
    except TypeError:
        return None 

  
    
    
if __name__ == '__main__':
    app.run_server(port = 8052)

KeyboardInterrupt: 

In [ ]:
import flask

server = flask.Flask(__name__)
app = dash.Dash(__name__, server=server)
    
if __name__ != '__main__':
    app.run_server(port = 8053)

In [ ]:
from flask import Flask
app = Flask(__name__)

@app.route('/')
def hello_world():
    return 'Hello World!'

if __name__ == '__main__':
  app.run(host='0.0.0.0')

 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [24/Feb/2019 01:56:05] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Feb/2019 01:56:06] "GET /favicon.ico HTTP/1.1" 404 -
